In [ ]:
import pandas as pd
import glob

# Step 0: Compile NAV data from ssga.com (ensure files are downloaded and placed in this directory). 
# For example, https://www.ssga.com/us/en/individual/etfs/the-technology-select-sector-spdr-fund-xlk contains the NAV data for XLK.
path = "your_directory_path_here/"  # Update this path to your directory containing the Excel files


# Step 1: Load all NAV Excel files
nav_files = glob.glob(path + "navhist-us-en-*.xlsx")
nav_dfs = []

for filename in nav_files:
    # Extract ticker from filename: "navhist-us-en-xlk.xlsx" -> "XLK"
    ticker = filename.split("navhist-us-en-")[1].replace(".xlsx", "").upper()
    
    # Skip the first 3 rows, use row 3 as header
    df = pd.read_excel(filename, skiprows=3)
    
    # Convert Date column, but handle errors (skip non-date rows)
    df["Date"] = pd.to_datetime(df["Date"], format="%d-%b-%Y", errors='coerce')
    
    # Drop rows where Date is NaN (this removes disclaimer text)
    df = df.dropna(subset=["Date"])
    
    # Filter for dates from June 25, 2018 onwards
    df = df[df["Date"] >= "2018-06-25"]
    
    # Convert Total Net Assets to numeric, handling any text
    df["Total Net Assets"] = pd.to_numeric(df["Total Net Assets"], errors='coerce')
    
    df = df[["Date", "Total Net Assets"]].rename(columns={"Total Net Assets": f"{ticker}_AUM"})
    
    # Drop any rows where AUM/TNA is NaN
    df = df.dropna()
    
    nav_dfs.append(df)
    print(f"✓ Loaded {ticker} ({len(df)} rows)")

# Step 2: Merge all AUM/TNA data
aum_merged = nav_dfs[0]
for df in nav_dfs[1:]:
    aum_merged = pd.merge(aum_merged, df, on="Date", how="outer")

aum_merged = aum_merged.sort_values("Date").reset_index(drop=True)

# Step 3: Calculate weights
tickers = ['XLK', 'XLF', 'XLV', 'XLY', 'XLP', 'XLE', 'XLI', 'XLU', 'XLRE', 'XLB', 'XLC']

aum_cols = [f"{ticker}_AUM" for ticker in tickers if f"{ticker}_AUM" in aum_merged.columns]
aum_merged['Total_AUM'] = aum_merged[aum_cols].sum(axis=1)

for ticker in tickers:
    if f"{ticker}_AUM" in aum_merged.columns:
        aum_merged[f'{ticker}_weight'] = aum_merged[f'{ticker}_AUM'] / aum_merged['Total_AUM']

# Save dataset
output_path = "/Users/drewrogers/Desktop/ETFsAndOT/"
aum_merged.to_csv(output_path + "etf_nav_and_weights.csv", index=False)

print("\n✅ Done! File saved as 'etf_nav_and_weights.csv'")
print(f"Shape: {aum_merged.shape}")
print(f"\nColumns: {list(aum_merged.columns)}")
print(f"\nSample:")
print(aum_merged.tail(3))
print(f"\nLatest weights (sum should = 1.0):")
weight_cols = [f'{ticker}_weight' for ticker in tickers if f'{ticker}_weight' in aum_merged.columns]
print(aum_merged[['Date'] + weight_cols].tail(1))
print(f"\nWeight sum check: {aum_merged[weight_cols].iloc[-1].sum():.6f}")